In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.11.0


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 0s 0us/step


## Build a machine learning model

Build a tf.keras.Sequential model

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

Sequential is useful for stacking layers where each layer has one input tensor and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the Flatten, Dense, and Dropout layers.

For each example, the model returns a vector of logits or log-odds scores, one for each class.

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.40250826, -0.36432612,  0.1376875 , -0.16155913,  0.03977738,
         0.33669686,  0.6200862 , -0.14671908, -0.6623274 ,  0.11985822]],
      dtype=float32)

The tf.nn.softmax function converts these logits to probabilities for each class:

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.06575774, 0.06831706, 0.11286283, 0.08367386, 0.10233617,
        0.13771449, 0.18283278, 0.08492484, 0.05071178, 0.1108684 ]],
      dtype=float32)

It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output. 

Define a loss function for training using losses.SparseCategoricalCrossentropy:

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.math.log(1/10) ~= 2.3.

In [7]:
loss_fn(y_train[:1], predictions).numpy()

1.9825727

Before you start training, configure and compile the model using Keras Model.compile. Set the optimizer class to adam, set the loss to the loss_fn function you defined earlier, and specify a metric to be evaluated for the model by setting the metrics parameter to accuracy

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the Model.fit method to adjust your model parameters and minimize the loss:



In [9]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 8s 3ms/step - loss: 0.2918 - accuracy: 0.9148
Epoch 2/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1420 - accuracy: 0.9574
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1063 - accuracy: 0.9673
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0891 - accuracy: 0.9720
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0764 - accuracy: 0.9756


The Model.evaluate method checks the model's performance, usually on a validation set or test set.

In [10]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0701 - accuracy: 0.9771 - 968ms/epoch - 3ms/step


[0.07010649144649506, 0.9771000146865845]

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [11]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [12]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.67651998e-08, 8.45347525e-09, 2.76581795e-06, 1.75437614e-04,
        3.48611764e-11, 2.85047946e-07, 8.65501044e-14, 9.99816597e-01,
        1.94622430e-06, 3.02292688e-06],
       [8.66156391e-09, 3.73037858e-03, 9.96263444e-01, 6.13548582e-06,
        1.18059886e-15, 7.39998507e-09, 5.40608447e-08, 2.78212728e-13,
        2.41983127e-08, 7.57982002e-13],
       [2.44498523e-07, 9.99500990e-01, 2.33867504e-05, 4.75533170e-06,
        9.71161935e-05, 1.18997132e-05, 4.47495804e-05, 2.90993281e-04,
        2.33900519e-05, 2.41531507e-06],
       [9.99926567e-01, 1.14100185e-09, 5.86981378e-06, 1.10059105e-07,
        3.33476493e-07, 2.10639473e-06, 5.99512896e-05, 2.68407189e-06,
        2.98085894e-08, 2.35758830e-06],
       [2.61057465e-07, 3.96637070e-08, 9.43659302e-07, 4.80183360e-09,
        9.96130109e-01, 1.30123752e-07, 6.42277655e-07, 6.32650626e-05,
        4.60145664e-08, 3.80464364e-03]], dtype=float32)>